# Supervised classification

In the data.csv there are letters (uppercases and lowercases) and numbers, 28x28 pixels in a row format.

* First, you need to know which labels are which, meaning you need to visualize some data to realize which number labels represents a letter, or a number.

In [1]:
# Import libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

%matplotlib inline

import matplotlib.pyplot as plt

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier

In [2]:
# Your code here
letters = pd.read_csv('data_all.zip')

In [3]:
letters.shape

(116322, 785)

In [4]:
chars = letters.e.unique()

In [5]:
print(sorted(chars))

[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0, 21.0, 22.0, 23.0, 24.0, 25.0, 26.0, 27.0, 28.0, 29.0, 30.0, 31.0, 32.0, 33.0, 34.0, 35.0, 36.0, 37.0, 38.0, 39.0, 40.0, 41.0, 42.0, 43.0, 44.0, 45.0, 46.0, 47.0, 48.0, 49.0, 50.0, 51.0, 52.0, 53.0, 54.0, 55.0, 56.0, 57.0, 58.0, 59.0, 60.0, 61.0]


In [6]:
count = len(letters.e.unique())
print(count)

62


In [7]:
letters.head(1)

,e,1,2,3,4,5,6,7,8,9,...,775,776,777,778,779,780,781,782,783,784
0,36.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
letters.dtypes

e      float64
1      float64
2      float64
3      float64
4      float64
        ...   
780    float64
781    float64
782    float64
783    float64
784    float64
Length: 785, dtype: object

In [9]:
digits = letters[(letters.e >= 0) & (letters.e <= 9)]

In [10]:
upper = letters[(letters.e >= 10) & (letters.e <= 37)]

In [11]:
lower = letters[(letters.e >= 37)]

In [12]:
#X = letters.drop('e', axis=1)
#y = letters['e']

In [13]:
#X = np.array(X)
#y = np.array(y)

* Try to do the same thing with numbers.

In [14]:
X_digit = digits.drop('e', axis=1)
y_digit = digits['e']

In [15]:
# Your code here

X_trainD, X_testD, y_trainD, y_testD = train_test_split(X_digit, y_digit, test_size=.20)

In [16]:
#Decision Tree
tree_model = DecisionTreeClassifier()
tree_model.fit(X_trainD, y_trainD)
print(tree_model.score(X_trainD,y_trainD))
print(tree_model.score(X_testD,y_testD))

1.0
0.8838052486187845


In [17]:
#Random forest
rfc = RandomForestClassifier()
rfc.fit(X_trainD,y_trainD)
print(rfc.score(X_trainD,y_trainD))
print(rfc.score(X_testD,y_testD))

1.0
0.9736705801104972


In [18]:
#GaussianNB
nb = GaussianNB()
nb.fit(X_trainD,y_trainD)
print(nb.score(X_trainD,y_trainD))
print(nb.score(X_testD,y_testD))

0.6527388095135321
0.6559046961325967


In [19]:
#Lo intenté correr con Logistic Regression & KNN pero tardó demasiado

* Try to do the same thing with lowercases.

In [20]:
# Your code here
X_lower = lower.drop('e', axis=1)
y_lower = lower['e']

In [21]:
X_trainL, X_testL, y_trainL, y_testL = train_test_split(X_lower, y_lower, test_size=.20)

In [22]:
tree_model = DecisionTreeClassifier()
tree_model.fit(X_trainL, y_trainL)
print(tree_model.score(X_trainL,y_trainL))
print(tree_model.score(X_testL,y_testL))

1.0
0.7251623057249655


In [23]:
rfc = RandomForestClassifier()
rfc.fit(X_trainL,y_trainL)
print(rfc.score(X_trainL,y_trainL))
print(rfc.score(X_testL,y_testL))

0.9999508164469801
0.8790084595711194


In [24]:
nb = GaussianNB()
nb.fit(X_trainL,y_trainL)
print(nb.score(X_trainL,y_trainL))
print(nb.score(X_testL,y_testL))

0.44004524886877827
0.43104465866614206


* Now, try to train a classifier model to predict the uppercases. Use every single model you know for classification.

In [ ]:
# Your code here

In [25]:
X_upper = upper.drop('e', axis=1)
y_upper = upper['e']

In [26]:
X_trainU, X_testU, y_trainU, y_testU = train_test_split(X_upper, y_upper, test_size=.20)

In [27]:
tree_model = DecisionTreeClassifier()
tree_model.fit(X_trainU, y_trainU)
print(tree_model.score(X_trainU,y_trainU))
print(tree_model.score(X_testU,y_testU))

1.0
0.719160880484562


In [28]:
rfc = RandomForestClassifier()
rfc.fit(X_trainU,y_trainU)
print(rfc.score(X_trainU,y_trainU))
print(rfc.score(X_testU,y_testU))

1.0
0.9033830698773821


In [29]:
nb = GaussianNB()
nb.fit(X_trainU,y_trainU)
print(nb.score(X_trainU,y_trainU))
print(nb.score(X_testU,y_testU))

0.48727514497839175
0.48072093366819324


In [ ]:
# En los tres casos ganó Random Forest!!!!